In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import preprocessingWithMissingvalues

carDf= preprocessingWithMissingvalues.preprocess('../ann/cardata.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
Month                   15420 non-null object
WeekOfMonth             15420 non-null int64
DayOfWeek               15420 non-null object
Make                    15420 non-null object
AccidentArea            15420 non-null object
DayOfWeekClaimed        15420 non-null object
MonthClaimed            15420 non-null object
WeekOfMonthClaimed      15420 non-null int64
Sex                     15420 non-null object
MaritalStatus           15420 non-null object
Age                     15420 non-null int64
Fault                   15420 non-null object
PolicyType              15420 non-null object
VehicleCategory         15420 non-null object
VehiclePrice            15420 non-null object
FraudFound              15420 non-null object
PolicyNumber            15420 non-null int64
RepNumber               15420 non-null int64
Deductible              15420 non-null int64
DriverRating           

/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;


In [4]:
carDateNormalized= carDf

In [5]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1
    

In [6]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 97)
(923, 97)


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNo,Deductible,DriverRating,DaysDiff
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.733333,0.00,0.000000,0.071795
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.933333,0.25,1.000000,0.069231
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.400000,0.25,0.666667,0.089744
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.200000,0.25,0.333333,0.107692
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.133333,0.25,0.000000,0.094872


In [7]:
#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


In [8]:
#save the preprocessed data
nonFraudulent.to_csv("nonFraudulentFeatures.csv",index=False,sep=",",header=columnlable)
fraudulent.to_csv("fraudulentFeatures.csv",sep=",",index=False,header=columnlable)
fraudulentLable.to_csv("fraudulentLable.csv",sep=",",index=False,header='Lable')
nonFraudulentLable.to_csv("nonFraudulentLable.csv",sep=",",index=False,header='Lable')


In [9]:
#nonFraudulent.head()
fraudulentLable.head()

,lable
28,1.0
52,1.0
53,1.0
94,1.0
96,1.0


In [10]:
#Note that for the train and test split, the lables and data have to be in numpy
nonFraudulentCarfeatures= nonFraudulent.values
nonFraudulentLable= nonFraudulentLable.values

fraudulentCarfeatures= fraudulent.values
fraudulentLable= fraudulentLable.values

print('non-Fraud: %d' %(nonFraudulentCarfeatures.shape[0]))
print('Fraud: %d' %(fraudulentCarfeatures.shape[0]))


non-Fraud: 14496
Fraud: 923


In [11]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulentCarfeatures,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulentCarfeatures, fraudulentLable,random_state=3,test_size=0.25)
print('fraudxtrain:',type(fraudX_train))
print(fraudX_test.shape)


nonFraudxtrain: <class 'numpy.ndarray'>
(3624, 97)
fraudxtrain: <class 'numpy.ndarray'>
(231, 97)


In [12]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [13]:
#make the  nonfraudulent dataset for train and test loader
myNonFraudulentTrainDataset= myDataset(nonFraudX_train,nonFraudY_train)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make the fraudulent dataset for train and test loader
myFraudulentTrainDataset= myDataset(fraudX_train,fraudY_train)
myFraudulentTestDataset= myDataset(fraudX_test,fraudY_test)

In [14]:
feature,lable= myNonFraudulentTrainDataset.__getitem__(0)
print("feature:",feature)
print("lable:",lable)
print(feature.size())

feature: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.2500, 0.0000, 0.0641])
lable: tensor([0.])
torch.Size([97])


In [15]:

#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

fraudulentTrainLoader= torch.utils.data.DataLoader(myFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
fraudulentTestLoader= torch.utils.data.DataLoader(myFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)



In [16]:
#Network architecture for the base autoencoders
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder= nn.Sequential(
            nn.Linear(97,70),
            nn.LeakyReLU(True),
            nn.Linear(70,60),
            nn.LeakyReLU(True),
            nn.Linear(60,45),
            nn.LeakyReLU(True),
            nn.Linear(45,20),nn.LeakyReLU(True),nn.Linear(20,10))
        
        self.decoder= nn.Sequential(
            nn.Linear(10,20),
            nn.LeakyReLU(True),
            nn.Linear(20,45),
            nn.LeakyReLU(True),
            nn.Linear(45,60),
            nn.LeakyReLU(True),
            nn.Linear(60,70),nn.LeakyReLU(True),nn.Linear(70,97),nn.ReLU())
    
    def forward(self,x):
        x= self.encoder(x)
        x= self.decoder(x)
        return x;

In [17]:
model= autoencoder()
criterion= nn.MSELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)

In [18]:
#train the model
for epoch in range(50):
    running_loss= 0.0
    for i, (feature1,lable1) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs1= torch.tensor(feature1)
        lables1= torch.tensor(lable1)
        lables1= lables.type(torch.LongTensor)
        
        # =====================forward====================
        output1 = model(inputs1)
        loss1 = criterion(output1,inputs1)
        
         # ===================backward====================
        optimizer.zero_grad()
        loss1.backward()
        optimizer.step()
        
        # =======print the statistics
        running_loss += loss.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss /nonFraudX_train.shape[0]))
      #running_loss = 0.0

        

NameError: name 'lables' is not defined

In [ ]:
 #test the model
def modelTest(Loader):
    lossList= []
    trueLable1= []
    for i,(feature1,lable1) in enumerate(Loader):
        inputs1= torch.tensor(feature1)
        output1= model(inputs1)
        loss1= criterion(output1,inputs1)
        trueLable1.append(lable)
        lossList.append(loss.item())
        
        if(i!=10):
            print("input1:",inputs1)
            print("output1:",output1)
            print("loss1:",loss1)
            print("lable:",lable1)
        else:
            assert(False)
        
    return lossList,trueLable1
        

In [ ]:
#test the model
lossList,trueLable1= modelTest(nonFraudulentTestLoader)
len(lossList)
lossList[1]

In [ ]:
len(trueLable1)
int(trueLable1[0])

In [ ]:
#Network architecture for the base autoencoders
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2,self).__init__()
        self.encoder2= nn.Sequential(
            nn.Linear(97,75),
            nn.LeakyReLU(True),
            nn.Linear(75,50),
            nn.LeakyReLU(True),
            nn.Linear(50,25),nn.LeakyReLU(True),nn.Linear(25,10))
       
        self.decoder2= nn.Sequential(
            nn.Linear(10,25),
            nn.LeakyReLU(True),
            nn.Linear(25,50),
            nn.LeakyReLU(True),
            nn.Linear(50,75),nn.LeakyReLU(True),nn.Linear(75,97),nn.ReLU())
    
    def forward(self,x):
        x= self.encoder2(x)
        x= self.decoder2(x)
        return x;

In [ ]:
model2= autoencoder2()
criterion2= nn.MSELoss()
optimizer2= torch.optim.SGD(model2.parameters(), lr=0.001,momentum=0.9)

In [ ]:
#train the model2
for epoch in range(100):
    running_loss2= 0.0
    for i, (feature2,lable2) in enumerate(fraudulentTestLoader):
        
        #gets the inputs
        inputs2= torch.tensor(feature2)
        lables2= torch.tensor(lable2)
        lables2= lables2.type(torch.LongTensor)
       
        # =====================forward====================
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        
        # ===================backward====================
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        # =======print the statistics
        running_loss2 += loss2.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss2 / fraudulentCarfeatures.shape[0]))
      #running_loss = 0.0

In [ ]:
def modelTest2(Loader2):
    lossList2= []
    trueLable1= []
    for i,(feature2,lable2) in enumerate(Loader2):
        inputs2= torch.tensor(feature2)
        output2= model2(inputs2)
        loss2= criterion(output2,inputs2)
        trueLable1.append(lable2)
        
        lossList2.append(loss2.item()) #put the loss
        if(i!=10):
            print("input2:",inputs2)
            print("output2:",output2)
            print("loss2:",loss2)
            print("lable2:",lable2)
        else:
            assert(False)
            
    return lossList2,trueLable1
        

In [ ]:
lossList2,trueLable1= modelTest2(nonFraudulentTestLoader)


In [ ]:
print("loss1:",lossList[1])
print("loss2:",lossList2[1])
len(trueLable1)

In [ ]:
predictedLable1=[]
for k in range(len(lossList)):
    if(lossList[k]<lossList2[k]):
        predictedLable1.append(0)
    else:
        predictedLable1.append(1)

In [ ]:

class1= predictedLable1.count(1)
class0= predictedLable1.count(0)
print('%d %d' %(class1,class0))
len(trueLable1)


In [ ]:
#test model1 with another type of data
lossList3,trueLable2= modelTest(fraudulentTestLoader)


In [ ]:
#test model2 with another type of data
lossList4,trueLable2= modelTest2(fraudulentTestLoader)


In [ ]:
predictedLable2=[]
for k in range(len(lossList3)):
    if(lossList3[k]<lossList4[k]):
        predictedLable2.append(0)
    else:
        predictedLable2.append(1)

In [ ]:
class1= predictedLable2.count(1)
class0= predictedLable2.count(0)
print('%d %d' %(class1,class0))


In [ ]:
totalLength1= len(trueLable1)
TP=FP=FN=TN= 0
for i in range(totalLength1):
    if(int(trueLable1[i])==1 and predictedLable1[i]==1):
        TP += 1
    elif(int(trueLable1[i])==1 and predictedLable1[i]==0):
        FN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==0):
        TN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==1):
        FP += 1

totalLength2= len(trueLable2)
for i in range(totalLength2):
    if(int(trueLable2[i])==1 and predictedLable2[i]==1):
        TP += 1
    elif(int(trueLable2[i])==1 and predictedLable2[i]==0):
        FN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==0):
        TN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==1):
        FP += 1
        
print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)
print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)
